In [1]:
import numpy as np
import pandas as pd

C:\Users\EXCELL  COMPUTERS\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\EXCELL  COMPUTERS\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
books = pd.read_csv('Books.csv', low_memory=False)
users = pd.read_csv('users.csv')
ratings = pd.read_csv('Ratings.csv')

In [5]:
books.sample(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
166464,0816633614,Jailhouse Stories: Memories of a Smalltown She...,Neil Haugerud,1999,University of Minnesota Press,http://images.amazon.com/images/P/0816633614.0...,http://images.amazon.com/images/P/0816633614.0...,http://images.amazon.com/images/P/0816633614.0...
149209,1900924242,A Journey Through America With the Rolling Stones,Robert Greenfield,2001,Helter Skelter Publishing,http://images.amazon.com/images/P/1900924242.0...,http://images.amazon.com/images/P/1900924242.0...,http://images.amazon.com/images/P/1900924242.0...
204590,0897932714,Alternative Treatments for Fibromyalgia &amp; ...,Mari Skelly,1999,Hunter House (CA),http://images.amazon.com/images/P/0897932714.0...,http://images.amazon.com/images/P/0897932714.0...,http://images.amazon.com/images/P/0897932714.0...
165682,0880295287,Dream and the Tomb a History of the Crusad,Robert Payne,0,Dorset Press,http://images.amazon.com/images/P/0880295287.0...,http://images.amazon.com/images/P/0880295287.0...,http://images.amazon.com/images/P/0880295287.0...
46714,0399131213,Break in,Dick Francis,1986,Putnam Pub Group,http://images.amazon.com/images/P/0399131213.0...,http://images.amazon.com/images/P/0399131213.0...,http://images.amazon.com/images/P/0399131213.0...


In [6]:
users.sample(5)

,User-ID,Location,Age
19915,19916,"dublin, dublin, ireland",30.0
82941,82942,"lake forest, california, usa",NaN
12030,12031,"oldenburg, niedersachsen, germany",25.0
151044,151045,"montreal, quebec, canada",27.0
164127,164128,"tampa, florida, usa",NaN


In [7]:
ratings.sample(5)

,User-ID,ISBN,Book-Rating
907529,220401,0345384369,8
753575,182183,0373254849,0
156209,35857,0312420323,0
1082036,258920,3404161335,0
872626,211344,0449006581,8


In [8]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 8)
(278858, 3)
(1149780, 3)


In [9]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [10]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [11]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [12]:
books.duplicated().sum()

0

In [13]:
users.duplicated().sum()

0

In [14]:
ratings.duplicated().sum()

0

## Popularity Based Recommender System

In [15]:
ratings_with_name = ratings.merge(books, on = 'ISBN')

In [16]:
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace = True)

In [17]:
num_rating_df.sample(3)

,Book-Title,num_ratings
107566,"Licks of Love: Short Stories and a Sequel, Rab...",5
232753,"When Megan Smiles (Harlequin American Romance,...",1
209746,The Tanaiste,2


In [19]:
# Convert 'Book-Rating' column to numeric type
ratings_with_name['Book-Rating'] = pd.to_numeric(ratings_with_name['Book-Rating'], errors='coerce')

# Group by 'Book-Title' and calculate mean of 'Book-Rating'
avg_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()

# Rename 'Book-Rating' to 'avg_ratings'
avg_rating_df.rename(columns={'Book-Rating': 'avg_ratings'}, inplace=True)


In [20]:
#avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
#avg_rating_df.rename(columns={'Book-Rating':'avg_ratings'},inplace = True)

In [21]:
avg_rating_df.head(5)

,Book-Title,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [22]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df.head(3)

,Book-Title,num_ratings,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,4,2.25
1,Always Have Popsicles,1,0.00
2,Apple Magic (The Collector's series),1,0.00


In [23]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_ratings',ascending=False).head(50)

In [24]:
popular_df

,Book-Title,num_ratings,avg_ratings
80434,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
80422,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
80441,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
80426,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
80414,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
191612,The Hobbit : The Enchanting Prelude to The Lor...,281,5.007117
187377,The Fellowship of the Ring (The Lord of the Ri...,368,4.948370
80445,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.895652
211384,"The Two Towers (The Lord of the Rings, Part 2)",260,4.880769
219741,To Kill a Mockingbird,510,4.700000


In [25]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','num_ratings','Image-URL-M','avg_ratings']]

In [26]:
popular_df.head(10)

,Book-Title,Book-Author,num_ratings,Image-URL-M,avg_ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,428,http://images.amazon.com/images/P/0439136350.0...,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,387,http://images.amazon.com/images/P/0439139597.0...,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,278,http://images.amazon.com/images/P/0590353403.0...,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,347,http://images.amazon.com/images/P/043935806X.0...,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,556,http://images.amazon.com/images/P/0439064872.0...,5.183453
16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,281,http://images.amazon.com/images/P/0345339681.0...,5.007117
17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,368,http://images.amazon.com/images/P/0345339703.0...,4.948370
26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,575,http://images.amazon.com/images/P/059035342X.0...,4.895652
28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,260,http://images.amazon.com/images/P/0345339711.0...,4.880769
39,To Kill a Mockingbird,Harper Lee,510,http://images.amazon.com/images/P/0446310786.0...,4.700000


## Collaborative Filtering Based Recommender System

In [27]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating']>200
padhe_likhe_users = x[x].index

In [28]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]

In [29]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [30]:
final_rating = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [31]:
pt = final_rating.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [32]:
pt.fillna(0,inplace=True)

In [33]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
similarity_score = cosine_similarity(pt)

In [36]:
similarity_score.shape

(706, 706)

In [37]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:6]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
    
    return data

In [38]:
import pickle

In [39]:
pickle.dump(popular_df,open('popular.pkl','wb'))

In [40]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_score,open('similarity_scores.pkl','wb'))